In [1]:
import dask.dataframe as dd
import numpy as np
import dask
from dask_ml.preprocessing import OneHotEncoder
from dask_ml.impute import SimpleImputer
from dask_ml.preprocessing import StandardScaler
from dask_ml.compose import ColumnTransformer

# Initial DF Setup

In [2]:
headers_orig = open('headers_orig.txt', 'r')
headers_orig = headers_orig.read().strip().split('\n')
orig = dd.read_csv("sample_orig_2018.txt", names=headers_orig, sep="|",sample=25000000)

headers_svcg = open('headers_svcg.txt', 'r')
headers_svcg = headers_svcg.read().strip().split('\n')
svcg = dd.read_csv("sample_svcg_2018.txt", names=headers_svcg, sep="|",sample=25000000)
svcg = svcg[['Loan Sequence Number', 'Zero Balance Code', 'Current Loan Delinquency Status']]
orig.head(10)

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator
0,784,201803,9,204802,49180.0,0,1,I,75,39,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N
2,731,201803,9,204802,NaN,0,1,P,45,37,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
3,763,201802,Y,204801,NaN,30,1,P,95,34,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
4,755,201803,9,204802,49700.0,0,1,P,80,33,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
5,728,201803,9,204802,NaN,0,1,P,80,44,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
6,690,201803,9,204802,NaN,0,1,P,73,34,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
7,691,201803,9,204802,24660.0,0,1,P,79,30,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N
8,671,201803,9,204802,22420.0,0,1,P,75,15,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
9,705,201803,Y,204802,17820.0,30,1,P,95,9,...,360,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N


In [3]:
orig = svcg.join(orig.set_index("Loan Sequence Number"), on="Loan Sequence Number", how="left")
orig.head(10)

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


,Loan Sequence Number,Zero Balance Code,Current Loan Delinquency Status,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator
0,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
1,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
2,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
3,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
4,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
5,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
6,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
7,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
8,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
9,F18Q10000027,NaN,0,784,201803,9,204802,49180.0,0,1,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N


In [4]:
def current_loan_helper(val):
    try:
        if int(val) >= 3:
            return 1
        else:
            return 0
    except:
        return 1
    
def zero_balance_helper(val):
    target = [3,6,9]
    if val in target:
        return 1
    else:
        return 0

In [5]:
orig['Zero Balance Code'] = orig['Zero Balance Code'].apply(zero_balance_helper, meta=('Zero Balance Code', 'int64'))
orig['Current Loan Delinquency Status'] = orig['Current Loan Delinquency Status'].apply(current_loan_helper, meta=('Current Loan Delinquency Status', 'object'))
orig['Current Loan Delinquency Status'] = orig['Current Loan Delinquency Status'].astype('int64')

orig['Default'] = orig['Zero Balance Code'] + orig['Current Loan Delinquency Status']
orig['Default'] = orig['Default'].apply(lambda x: x>0, meta=('Default', 'bool'))
orig = orig.drop(['Current Loan Delinquency Status','Zero Balance Code'], axis=1)
#orig = orig.drop_duplicates(subset=['Loan Sequence Number'], keep='last')
orig.head(10)

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (7,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))
c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


,Loan Sequence Number,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),...,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator,Default
21,F18Q10000027,784,201803,9,204802,49180.0,0,1,I,75,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
48,F18Q10000052,693,201803,N,203302,24340.0,0,1,P,80,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N,False
74,F18Q10000083,731,201803,9,204802,NaN,0,1,P,45,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
109,F18Q10000116,763,201802,Y,204801,NaN,30,1,P,95,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
144,F18Q10000139,755,201803,9,204802,49700.0,0,1,P,80,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
159,F18Q10000192,728,201803,9,204802,NaN,0,1,P,80,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
194,F18Q10000254,690,201803,9,204802,NaN,0,1,P,73,...,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
223,F18Q10000259,691,201803,9,204802,24660.0,0,1,P,79,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N,False
254,F18Q10000269,671,201803,9,204802,22420.0,0,1,P,75,...,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,False
289,F18Q10000270,705,201803,Y,204802,17820.0,30,1,P,95,...,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N,False


# Preprocessing

In [6]:
#FEATURES TO DROP
drop_features = ['First Payment Date', 
                 'Maturity Date', 
                 'Metropolitan Statistical Area (MSA) Or Metropolitan Division','Pre-HARP Loan Sequence Number',
                 'Property State',
                 'Postal Code',
                 'Loan Sequence Number',
                 'Seller Name',
                 'Servicer Name',
                 'Pre-HARP Loan Sequence Number',
                 'Property Valuation Method']
orig = orig.drop(drop_features, axis=1)
orig = orig[orig['HARP Indicator'] != 'Y']
orig = orig.drop('HARP Indicator', axis=1)

In [7]:
#FEATURES TO IMPUTE
simpleImputer = SimpleImputer(missing_values=99, strategy='constant', fill_value=1)
orig['Number of Units'] = simpleImputer.fit_transform(orig[['Number of Units']])['Number of Units']

simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='P')
orig['Occupancy Status'] = simpleImputer.fit_transform(orig[['Occupancy Status']])['Occupancy Status']

simpleImputer = SimpleImputer(missing_values=999, strategy='constant', fill_value=65)
orig['Original Debt-to-Income (DTI) Ratio'] = simpleImputer.fit_transform(orig[['Original Debt-to-Income (DTI) Ratio']])['Original Debt-to-Income (DTI) Ratio']

simpleImputer = SimpleImputer(missing_values=999, strategy='constant', fill_value=np.NaN)
orig['Original Loan-to-Value (LTV)'] = simpleImputer.fit_transform(orig[['Original Loan-to-Value (LTV)']])['Original Loan-to-Value (LTV)']
simpleImputer = SimpleImputer(strategy='mean', fill_value=1)
orig['Original Loan-to-Value (LTV)'] = simpleImputer.fit_transform(orig[['Original Loan-to-Value (LTV)']])['Original Loan-to-Value (LTV)']

simpleImputer = SimpleImputer(missing_values='T', strategy='constant', fill_value='R')
orig['Channel'] = simpleImputer.fit_transform(orig[['Channel']])['Channel']
simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='R')
orig['Channel'] = simpleImputer.fit_transform(orig[['Channel']])['Channel']

simpleImputer = SimpleImputer(missing_values=9, strategy='constant', fill_value=1)
orig['Prepayment Penalty Mortgage (PPM) Flag'] = simpleImputer.fit_transform(orig[['Prepayment Penalty Mortgage (PPM) Flag']])['Prepayment Penalty Mortgage (PPM) Flag']

simpleImputer = SimpleImputer(missing_values='99', strategy='constant', fill_value='SF')
orig['Property Type'] = simpleImputer.fit_transform(orig[['Property Type']])['Property Type']

simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='P')
orig['Loan Purpose'] = simpleImputer.fit_transform(orig[['Loan Purpose']])['Loan Purpose']

simpleImputer = SimpleImputer(missing_values=9, strategy='constant', fill_value=1)
orig['Number of Borrowers'] = simpleImputer.fit_transform(orig[['Number of Borrowers']])['Number of Borrowers']

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (7,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))
c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


In [8]:
#FEATURES TO BINARIZE
orig['Prepayment Penalty Mortgage (PPM) Flag'] = orig['Prepayment Penalty Mortgage (PPM) Flag'].replace({'Y': 1, 'N': 0})

orig['Amortization Type (Formerly Product Type)'] = orig['Amortization Type (Formerly Product Type)'].replace({'FRM': 1, 'ARM': 0})

simpleImputer = SimpleImputer(strategy='constant', fill_value='N')
orig['Super Conforming Flag'] = simpleImputer.fit_transform(orig[['Super Conforming Flag']])['Super Conforming Flag']
orig['Super Conforming Flag'] = orig['Super Conforming Flag'].replace({'Y': 1, 'N': 0})

orig['Program Indicator'] = orig['Program Indicator'].replace({'H': 1, '9': 0})

orig['Interest Only (I/O) Indicator'] = orig['Interest Only (I/O) Indicator'].replace({'Y': 1, 'N': 0})

In [9]:
onehot_features = ['First Time Homebuyer Flag', 
                   'Occupancy Status', 
                   'Channel',
                   'Property Type',
                   'Loan Purpose']

orig['First Time Homebuyer Flag'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Occupancy Status'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Channel'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Property Type'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Loan Purpose'] = orig['First Time Homebuyer Flag'].astype("category")
orig = orig.categorize(columns=['First Time Homebuyer Flag','Occupancy Status','Channel','Property Type','Loan Purpose'])

onehot_transformer = OneHotEncoder()

numerical_features = ['Credit Score', 
                      'Mortgage Insurance Percentage (MI %)', 
                      'Number of Units', 
                      'Original Combined Loan-to-Value (CLTV)', 
                      'Original Debt-to-Income (DTI) Ratio', 
                      'Original UPB', 
                      'Original Loan-to-Value (LTV)',
                      'Original Interest Rate',
                      'Original Loan Term',
                      'Number of Borrowers']
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(transformers=[
    ('onehot', onehot_transformer, onehot_features), 
    ('numerical', numerical_transformer, numerical_features)],
    remainder='passthrough'
)

orig = preprocessor.fit_transform(orig)
orig.head(10)

c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:151: DtypeWarning: Columns (7,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, cache)
c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:151: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, cache)
c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (7,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))
c:\users\hiroki\appdata\local\programs\python\python38\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (3,7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))
c:\users\hiroki\appdata\local\

,First Time Homebuyer Flag_9,First Time Homebuyer Flag_N,First Time Homebuyer Flag_Y,Occupancy Status_9,Occupancy Status_N,Occupancy Status_Y,Channel_9,Channel_N,Channel_Y,Property Type_9,...,Original Loan-to-Value (LTV),Original Interest Rate,Original Loan Term,Number of Borrowers,Prepayment Penalty Mortgage (PPM) Flag,Amortization Type (Formerly Product Type),Super Conforming Flag,Program Indicator,Interest Only (I/O) Indicator,Default
21,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-0.062355,-0.236655,0.416217,-0.925756,0,1,0,0,0,False
48,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.229357,-3.117368,-2.527024,1.080198,0,1,0,0,0,False
74,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-1.812628,-2.069836,0.416217,-0.925756,0,1,0,0,0,False
109,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.104494,-1.022304,0.416217,-0.925756,0,1,0,0,0,False
144,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.229357,-1.807953,0.416217,1.080198,0,1,0,0,0,False
159,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.229357,-1.284187,0.416217,-0.925756,0,1,0,0,0,False
194,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-0.179040,-1.284187,0.416217,-0.925756,0,1,0,0,0,False
223,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.171015,-0.498538,0.416217,1.080198,0,1,0,0,0,False
254,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-0.062355,-0.236655,0.416217,1.080198,0,1,0,0,0,False
289,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.104494,-1.807953,0.416217,1.080198,0,1,0,0,0,False
